# NARMA10 task with Echo State Networks

This task consists in predicting the output of a 10-th order non-linear autoregressive moving average (NARMA) system.
- [Reference paper](https://doi.org/10.1016/j.neunet.2011.02.002)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import itertools

from sklearn.metrics import mean_squared_error
# from gridsearch import GS

The input of the system is a sequence of elements 𝑥(𝑡) randomly chosen according to a uniform distribution over [0, 0.5]. 

Given the input value 𝑥(𝑡), the task is to predict the corresponding value of 𝑦(𝑡).

- Import the dataset from the .csv file *NARMA10.csv*, where the first row represents the input and the second row represents the
target output. Different columns represent different time-steps.
- Split the data into training (the first 5000 time steps), and test set (remaining time steps). Note that for model selection you will use the data in the training set, with a further split in training (first 4000 samples) and validation (last 1000 samples).
    - For the sake of problem understanding, you can try to first visualize the timeseries data

In [ ]:
# dset = pd.read_csv('../data/NARMA10.csv').to_numpy()


In [ ]:
dset = pd.read_csv('../data/NARMA10.csv', header=None).to_numpy()

x = dset[0]
y = dset[1]
x_test = x[:5000]
train = x[5000:] 

x_train = train[:4000]
x_val = train[4000:]

y_test = y[:5000]
train_y = y[5000:] 

y_train = train_y[:4000]
y_val = train_y[4000:]

In [ ]:
# plot_ts(x_train,x_val,x_test, "NARMA10 input values")

limit1=int(x_train.shape[0])
limit2=int(x_train.shape[0]+x_val.shape[0])
limit3=int(limit2+x_test.shape[0])

plt.figure(figsize=(12,5), dpi=200)
plt.plot(range(0,limit1), x_train, color="indigo", linewidth=0.2, label="training set") 
plt.plot(range(limit1,limit2), x_val, color="purple", linewidth=0.2, label="validation set") 
plt.plot(range(limit2,limit3), x_test, color="violet", linewidth=0.2, label="test set") 

plt.title('NARMA10 input values')
plt.xlabel("Time steps")
plt.ylabel("Values")
plt.legend()
plt.grid(color = 'green', linestyle = '--', linewidth = 0.2)
#plt.savefig(str('plots/timeseries.jpeg'), edgecolor='black', dpi=400, transparent=True)
plt.show()

Implement from scratch the code required to initialize, run, train, and evaluate an **Echo State Network**. Your implementation should take into
consideration relevant hyper-parametrization of the neural network (e.g., number of reservoir neurons, spectral radius, etc.)

#### Grid search

In [ ]:

###############################################################################
# # first test the nn then resume this complex grid
###############################################################################


class GS:
    
    def __init__(self, parameters:dict, Xset:tuple, Yset:tuple):
        self.Xset=Xset
        self.Yset=Yset
        param_grid = self.grid(self, parameters)
        min_loss, best_model = self.search(self, param_grid)
        self.min_loss = min_loss
        self.best_model = best_model # the best model discovered

    
    @staticmethod
    def grid(self, params):
        param_names=list(params.keys())
        param_values=list(params.values())
        param_combinations=list(itertools.product(*param_values))
        
        param_grid=[]
        for combination in param_combinations:
            param_grid.append(dict(zip(param_names, combination)))
        return param_grid 
        
        ###       !! FROM HERE ON, TO BE CHECKED BETTER AFTER DEVELOPING THE TRAINING FUNCION !!      ###
    @staticmethod
    def search(self, param_grid:tuple):#, Xset, Yset):
        predictions={}  
        for pg in param_grid:
            model=ESN(
                input_dim=self.Xset.shape[1], 
                reservoir_dim=pg['reservoir'], 
                omega=pg['omega'], 
                sr=pg['sr'], 
                sparsity=pg['sparsity'], 
                leak=pg['leak']
                )
            tr_loss, y_pred_tr=model.fit(self.Xset, self.Yset) #also do the predict that actually returns what's requested here
            val_loss, y_pred_val=model.predict(self.Xset, self.Yset,y_pred_tr[-1])
            predictions[loss]=[pg, y_pred] 
        
        # predictions=sorted(predictions.items())
        # min_loss=list(predictions.keys())[0]
        # best_model=list(predictions.values())[0]
        min_loss = min(predictions.keys())
        best_model = predictions[min_loss]
        
        return min_loss, best_model

## Echo State Network

In [ ]:
'''
This code defines a class called `ESN` which stands for Echo State Network. 
An Echo State Network is a type of recurrent neural network known for its simplicity and effectiveness in time-series prediction tasks.
'''
class ESN:
    def __init__(self, input_dim, reservoir_dim, omega, sr, sparsity, leak):
        self.input_dim = 1
        self.reservoir_dim = reservoir_dim
        self.output_dim = 1
        self.omega = omega
        self.sr = sr
        self.sparsity = sparsity
        self.leak = leak
        self.bias = (np.random.rand(reservoir_dim, 1) * 2 - 1) * omega
        self.transient=100

        # weight initialization
        ##W IN WITH SECOND DIMENSION AS INPUT_DIM IS A TEST
        self.W_in = (np.random.rand(reservoir_dim,input_dim) * 2 - 1) * omega #Choose the values of W from a random distribution scaled by a hyper-parameter ω_in
        
        self.W_res = np.random.rand(reservoir_dim, reservoir_dim) * 2 - 1 
        # spectral radius (ro) condition
        radius = np.max(np.abs(np.linalg.eigvals(self.W_res)))
        self.W_res *= sr / radius
        # apply sparsity
        mask = np.random.rand(reservoir_dim, reservoir_dim) > self.sparsity
        self.W_res[mask] = 0
        
        self.W_out = np.random.rand(reservoir_dim, self.output_dim) * 2 - 1
        
        self.state = np.zeros(reservoir_dim)
        
        
    def _update_state(self, input_vector):
        pre_activation = np.dot(self.W_in, input_vector) + np.dot(self.W_res, self.state) + self.bias 

        self.state = np.tanh(pre_activation + self.leak * (np.random.rand(self.reservoir_dim) - 0.5))
        return self.state
    
    
    def fit(self, inputs, target):
        states = []
        # states.append(self.state)
        
        # print('INPUTS:', inputs)
        for input_vector in inputs:
            states.append(self._update_state(input_vector))
        
        # states = np.vstack(states)
        # Add bias term
        # states = np.hstack((states, np.ones((states.shape[0], 1))))
        # print(f'target: {target}, states: {states}')

        # Discard the transient
        # states, target = states[self.transient:], target[self.transient:]
        
        # Ridge regression (Tikhonov regularization)
        reg = 1e-8
        #readout
        self.W_out = np.dot(np.dot(target.T, states), np.linalg.inv(np.dot(states.T, states) + reg * np.eye(states.shape[1]))).T
        
        y_pred = states @ self.W_out
        return mean_squared_error(target, y_pred), states[-1]

        
    def predict(self, inputs,target,h0):
        states = []
        states.append(h0)

        for input_vector in inputs:
            states.append(self._update_state(input_vector))
        # states.append(self._update_state(inputs))
        
        # states = np.vstack(states)
        # Add bias term
        # states = np.hstack((states, np.ones((states.shape[0], 1))))
        
        y_pred=np.dot(states, self.W_out)
        loss = mean_squared_error(target, y_pred)

        output = states[-1], y_pred
        return ((loss,) + output) #if loss is not None else output


In [ ]:
Xtrain = x_train.reshape(-1, 1) #.reshape(-1, 1, 1)
Ytrain = y_train.reshape(-1, 1)
Xtest = x_test.reshape(-1, 1) #.reshape(-1, 1, 1)
Ytest = y_test.reshape(-1, 1)

Xtrain.shape, Ytrain.shape, Xtest.shape, Ytest.shape

In [ ]:
params={
    'reservoir':[100, 200, 500],
    'omega':[0.1, 0.5],
    'sr':[0.5, 0.8, 0.9],
    'sparsity':[0.5, 0.8, 0.9],
    'leak':[0.01, 0.001, 0.0001]
}

grid_search=GS(params, Xtrain, Ytrain) 
best_loss, best_model=grid_search.min_loss, grid_search.best_model